<img style="float: right; margin: 5px 5px 20px 20px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="100px" height="75px"/>

# 002 Pairs Trading

### Microstructures and trading systems

> **Evelin Ramirez, Pedro Gael Rayas**

## Overview
Este proyecto implementa una estrategia de Pairs Trading basada en el análisis de series temporales cointegradas. La idea central es identificar dos activos financieros que presenten una relación a largo plazo estable, de manera que la combinación lineal de sus precios sea estacionaria. Para ajustar dinámicamente la relación entre ambos activos, se utiliza un Filtro de Kalman implementado desde cero, lo que permite estimar en tiempo real el hedge ratio (razón de cobertura). Las señales de trading se generan cuando el spread (diferencia entre el precio del activo dependiente y el ajustado por el hedge ratio del activo independiente) se desvía significativamente de su media, utilizando umbrales basados en desviaciones estándar. Finalmente, la estrategia se evalúa mediante un proceso de backtesting que simula operaciones en una cuenta de margen con comisiones y un capital inicial definido.


## Introduction
La creciente competencia en los mercados financieros ha llevado a la búsqueda de estrategias que permitan obtener beneficios a través del arbitraje estadístico. Una de estas estrategias es el Pairs Trading, la cual se basa en la identificación de pares de activos que, a pesar de sufrir fluctuaciones a corto plazo, mantienen una relación estable en el largo plazo.

El presente proyecto se enfoca en la implementación de una estrategia de Pairs Trading mediante:

* La verificación de cointegración entre los activos, lo que asegura que la combinación lineal de sus precios es estacionaria.
* La estimación dinámica del hedge ratio mediante un Filtro de Kalman implementado desde cero, permitiendo ajustar la exposición a cada activo conforme evoluciona el mercado.
* La generación de señales de trading a partir de desviaciones significativas del spread respecto a su media, aplicando umbrales de ±1.5 desviaciones estándar para entrar en posición y señales de salida cuando el spread regresa a niveles cercanos a la media.
* El backtesting de la estrategia, en el cual se simula la ejecución de operaciones reales considerando un capital inicial de $1,000,000 USD, comisiones de 0.125% y el uso de cuentas de margen.

Esta aproximación permite no solo validar la viabilidad de la estrategia en términos históricos, sino también comprender mejor la dinámica del spread y la efectividad del filtro en la adaptación a cambios en la relación entre los activos.

## Methodology

La metodología adoptada en este proyecto se puede desglosar en las siguientes etapas:

1. **Descarga y Preprocesamiento de Datos:**
   - Se obtienen 10 años de datos históricos de los activos seleccionados (por ejemplo, AAPL y MSFT) mediante una fuente confiable.
   - Los datos se limpian y se convierten a formatos adecuados (por ejemplo, asegurando que los precios sean numéricos y la fecha esté en formato `datetime`).

2. **Verificación de Cointegración:**
   - Se realiza un test de cointegración (por ejemplo, el test de Johansen) para determinar si la combinación lineal de los precios de los dos activos es estacionaria.
   - Esta etapa es fundamental para justificar que existe una relación a largo plazo entre los activos, lo que respalda la premisa del Pairs Trading.

3. **Estimación del Hedge Ratio mediante Filtro de Kalman:**
   - Se implementa desde cero un Filtro de Kalman que permite estimar de forma dinámica el hedge ratio, es decir, la proporción en la que se deben combinar los activos para obtener una relación estacionaria.
   - El Filtro de Kalman se actualiza de forma recursiva, adaptándose a la evolución de los precios y permitiendo capturar cambios en la relación entre los activos.

4. **Cálculo del Spread y Generación de Señales de Trading:**
   - Con el hedge ratio estimado, se calcula el spread como la diferencia entre el precio del activo dependiente y el producto del hedge ratio por el precio del activo independiente.
   - Se definen umbrales basados en la media y la desviación estándar del spread:
     - **Long Signal:** cuando el spread cae por debajo de la media menos 1.5 desviaciones estándar.
     - **Short Signal:** cuando el spread supera la media más 1.5 desviaciones estándar.
     - **Exit Signal:** cuando el spread regresa a un rango cercano a la media (por ejemplo, dentro de ±0.5 desviaciones estándar).

5. **Backtesting de la Estrategia:**
   - Se simula la ejecución de operaciones a lo largo del tiempo utilizando las señales generadas, considerando un capital inicial de \$1,000,000 USD.
   - Se modelan las operaciones de apertura y cierre de posiciones, aplicando una comisión del 0.125% en cada transacción y, si es necesario, se simula el uso de cuentas de margen.
   - Se registra la evolución del capital (curva de equity) y se calculan métricas de rendimiento que permiten evaluar la efectividad de la estrategia.

6. **Visualización y Análisis de Resultados:**
   - Se generan gráficos que muestran la evolución del spread, las señales de trading y la curva de capital a lo largo del tiempo.
   - Estos gráficos y métricas permiten identificar períodos de alta rentabilidad, drawdowns y la robustez de la estrategia ante cambios en la dinámica de los precios.







## Code Implementation

In [5]:
#import yfinance as yf
#import pandas as pd

#def download_data(tickers, start_date, end_date):
 #   data = yf.download(tickers, start=start_date, end=end_date)
  #  data.reset_index(inplace=True)  # Asegura que la fecha sea una columna
   # data.to_csv('historical_data.csv', index=False)  # Guarda correctamente

#if __name__ == "__main__":
 #   tickers = ['AAPL', 'MSFT']  # Activos a analizar
  #  start_date = '2014-01-01'
   # end_date = '2024-01-01'
    # download_data(tickers, start_date, end_date)
    

In [6]:
import pandas as pd

file_path = "historical_data.csv"
data = pd.read_csv(file_path)

data_info = data.info()
data_head = data.head()

missing_values = data.isnull().sum()

columns = data.columns

data_info, data_head, missing_values, columns

#

data_cleaned = data.iloc[2:].reset_index(drop=True)

numeric_columns = ['Close', 'Close.1', 'High', 'High.1', 'Low', 'Low.1', 'Open', 'Open.1', 'Volume', 'Volume.1']
data_cleaned[numeric_columns] = data_cleaned[numeric_columns].astype(float)

data_cleaned = data_cleaned.dropna()

data_cleaned.head()

data_cleaned = data_cleaned.rename(columns={'Price': 'Date'})
data_cleaned['Date'] = pd.to_datetime(data_cleaned['Date'])

data_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         0 non-null      float64
 1   Adj Close    1 non-null      object 
 2   Adj Close.1  1 non-null      object 
 3   Close        1 non-null      object 
 4   Close.1      1 non-null      object 
 5   High         1 non-null      object 
 6   High.1       1 non-null      object 
 7   Low          1 non-null      object 
 8   Low.1        1 non-null      object 
 9   Open         1 non-null      object 
 10  Open.1       1 non-null      object 
 11  Volume       1 non-null      object 
 12  Volume.1     1 non-null      object 
dtypes: float64(1), object(12)
memory usage: 232.0+ bytes


,Date,Adj Close,Adj Close.1,Close,Close.1,High,High.1,Low,Low.1,Open,Open.1,Volume,Volume.1


In [ ]:
!pip uninstall -y yfinance


In [7]:
import numpy as np
import pandas as pd
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from kalman_filter import KalmanFilter

def check_cointegration(data):
    result = coint_johansen(data, det_order=0, k_ar_diff=1)
    return result.lr1[0] > result.cvt[0, 0]  # Check if cointegrated

def calculate_hedge_ratio(data):
    kf = KalmanFilter()
    hedge_ratios = []

    for x, y in zip(prices_x, prices_y):
        kf.predict()
        kf.update(x, y)
        hedge_ratios.append(kf.get_hedge_ratio())

    return np.array(hedge_ratios)

In [11]:
import yfinance as yf
import pandas as pd

# Definir los tickers y el rango de fechas
tickers = ['AAPL', 'MSFT']
start_date = '2014-01-01'
end_date = '2024-01-01'

# Descargar los datos históricos con yfinance
data_downloaded = yf.download(tickers, start=start_date, end=end_date)

# Reiniciar el índice para que la fecha se convierta en una columna
data_downloaded.reset_index(inplace=True)

# Guardar los datos en un archivo CSV
data_downloaded.to_csv("historical_data.csv", index=False)

print("Datos descargados y guardados en 'historical_data.csv'")


[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
- AAPL: No data found for this date range, symbol may be delisted
- MSFT: No data found for this date range, symbol may be delisted
Datos descargados y guardados en 'historical_data.csv'


In [10]:
import pandas as pd

# Cargar el CSV con los datos históricos
# Se asume que "historical_data.csv" contiene al menos las columnas: "Date", "AAPL" y "MSFT"
data = pd.read_csv("historical_data.csv")

# Convertir la columna de fecha a formato datetime
data['Date'] = pd.to_datetime(data['Date'])

# Asegurar que las columnas de precios sean numéricas
data['AAPL'] = pd.to_numeric(data['AAPL'], errors='coerce')
data['MSFT'] = pd.to_numeric(data['MSFT'], errors='coerce')

# Eliminar filas que tengan NaN en las columnas críticas
data = data.dropna(subset=['AAPL', 'MSFT', 'Date'])

# Mostrar las primeras filas
data.head()


KeyError: 'AAPL'

## Results


## Conclusions


## Github
